# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [52]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key
# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [53]:
# File to Load (Remember to Change These)
file_to_load = "cities.csv"

# Read Purchasing File and store into Pandas data frame
weather_data = pd.read_csv(file_to_load)
weather_data1=weather_data.dropna()
weather_data1

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [54]:
# Store latitude and longitude in locations
locations = weather_data1[["Lat", "Lng"]]
locations
# Fill NaN values and convert to float
Humidity = weather_data1["Humidity"].astype(float)
Humidity

0      72.0
1      74.0
2      73.0
3      39.0
4      76.0
       ... 
543    32.0
544     4.0
545    62.0
546    93.0
547    64.0
Name: Humidity, Length: 544, dtype: float64

In [55]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [56]:
# selecting units based on condition [A max temperature lower than 80 degrees but higher than 70.]
Max_Temp = weather_data1.loc[weather_data1["Max Temp"] > 70] 
Max_Temp1 = Max_Temp.loc[Max_Temp["Max Temp"] < 80]
print("A max temperature lower than 80 degrees but higher than 70.")
print(Max_Temp1)
print("---------------------------------------------------------------")

# selecting units based on condition [Wind speed less than 10 mph.]
Wind_speed = weather_data1.loc[weather_data1["Wind Speed"] < 10]
print("Wind speed less than 10 mph.")
print(Wind_speed)
print("---------------------------------------------------------------")

# selecting units based on condition [Zero cloudiness.]
Cloudiness = weather_data1.loc[weather_data1["Cloudiness"] == 0]
print("Zero cloudiness.")
print(Cloudiness)
print("---------------------------------------------------------------")

# selecting units based on condition [ideal weather]
ideal_weather = Max_Temp1.loc[Max_Temp1["Wind Speed"] < 10] 
ideal_weather1= ideal_weather.loc[ideal_weather["Cloudiness"] == 0]
print("ideal weather")
print(ideal_weather1)


A max temperature lower than 80 degrees but higher than 70.
     City_ID             City  Cloudiness Country        Date  Humidity  \
1          1        mahebourg          75      MU  1558378503        74   
12        12           avarua         100      CK  1558378756        94   
18        18  bambous virieux          75      MU  1558378758        74   
25        25          kantang          20      TH  1558378759        94   
28        28          rikitea          95      PF  1558378669        76   
..       ...              ...         ...     ...         ...       ...   
516      516          malindi          75      KE  1558378808        88   
521      521        high rock          75      BS  1558378893        83   
530      530           hirara          37      JP  1558378895        67   
534      534           marzuq          84      YE  1558378896        36   
539      539             palu          72      ID  1558378897        80   

       Lat     Lng  Max Temp  Wind Spee

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [81]:
# geocoordinates
# We can get all the information from above.
target_coordinates = ["-20.63,-46","-30.17,-50.22","-20.30,44.28","15.34,74.49","35.17,-2.93","22.92,96.51","32.86,59.22"]
target_search = "hotels"
target_radius = 5000


# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

# convert response to json
places_data = response.json()

# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))

# find all hotels in those ideal cities.
counter = 0
print("The first Hotel in the seven cities with ideal weathers")

for place in places_data["results"]:
    print(place["name"])
    print(place["vicinity"])
    print("--------------------------------")
    counter += 1
    if counter == len(target_coordinates):
        break

{
    "html_attributions": [],
    "next_page_token": "CqQCGwEAABdb2hUL4yL8_Ql8R3_HofWrEZugJn0HKOy8TwpmzgvWYTSKcCqR1CmDDZ5zZ-Q2UTTttMeRVYNFHIfUTNjbYVNXJ4C2N9Qz8gLprBtZTtJuaz3o5X0gfn2hoB3vhqOWbQvBn228EVipjevdPi_2BrueQoKN5FGP_Cicc3dmDH6YQzhQXnKhbbbccatg__Qbl_n-QGtuP4kfSChIkZu23tQ01QfWb7vcRDUt9LbAxEiyjMRVo4x3FL1fPyzigG6jR4zpOQ5myisyqNO7w-sydyz9nu1wZ0hGCv4_c2mqecBMwKkW0YMcocDt0I1AGZ78TpRUzRkGGMlorqYUgQXHIfcaPCOYa2q2BFszsdrKAmVmun4QP8MU56ViK49bpyXVzhIQGYTjlkRSp08pL5aRu3bPJBoUQnv99IIMSYI3HK38rwMnMRH0aQE",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.6425523,
                    "lng": -46.00563959999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.64119272010728,
                        "lng": -46.00434157010727
                    },
                    "southwest": {
                        "lat": -20.64

In [ ]:
hotel_df

In [58]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display figure
